# Process the raw data from pyapm and bpnsdata
This notebook is the code to process the output given after processing the data with pypam and bpnsdata
For more information about this process please contact clea.parcerisas@vliz.be or check the documentation of both packages
https://lifewatch-pypam.readthedocs.io/en/latest/
https://github.com/lifewatch/bpnsdata

In [1]:
# Install the required packages. Geopandas can give problems in Windows machines, so better to install them using wheels when using Windows
import sys
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install geopandas
!{sys.executable} -m pip install netCDF4
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install xarray

In [1]:
import datetime
import pathlib

import geopandas
import numpy as np
import pandas as pd
import xarray
from tqdm import tqdm

C:\Users\cleap\Envs\wadden_soundscape\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
ENV_LABELS = [
    'route_density',
    'season',
    'moon_phase',
    'day_moment',
    'sediment_type',
    'bathymetry',
]

# List all the labels which you have on your Raven file which are TO BE REMOVED (artifacts)
ARTIFACTS_LABELS = ['clipping', 'rope_noise']

CHUNK_LENGTH = 60.0 # This has to be the SAME one you chose in 0_create_dataset (binsize)

# Set the min and max frequencies to use (has to be a range smaller or equal to the one you selected in 0_create_dataset)
MAX_FREQ = 24000
MIN_FREQ = 60

In [11]:
# Write down which of the variables are CATEGORIES and not NUMERICAL
CATEGORICAL_VARS = ['day_moment', 'sediment_type', 'seabed_habitat', 'label']
CYCLIC_VARS = ['season', 'moon_phase']

vars_dtypes = {
    'route_density': int,
    'season': int,
    'moon_phase': np.float16,
    'day_moment': 'category',
    'sediment_type': 'category',
    'bathymetry': np.float16,
}


In [4]:
# Define the folders
data_path = pathlib.Path('./data/raw_data/')
processed_data_path = pathlib.Path('./data/processed/')
raw_data_path = pathlib.Path('./data/raw_data/deployments/')

In [5]:
# Read the metadata csv
metadata = pd.read_csv(data_path.joinpath('data_summary_mda.csv'))

# Read the labelled data
# Change the path depending on where your data is
# Change the names of the columns start_datetime, end_datetime and start_file to match the ones in the file
# Change the label column to the correct one!
label_column = 'tag'
labels_data = pd.read_csv(data_path.joinpath('labels.csv'), parse_dates=['start_datetime',
                                                                         'end_datetime',
                                                                         'start_file'])

# Create the empty output vars
df_features = pd.DataFrame()
df_sample = pd.DataFrame()
df_env = pd.DataFrame()
df_geo = geopandas.GeoDataFrame()
df_labels = pd.DataFrame()

# Define the names of the vars that will be used
features_var = 'oct3'
freqticks = None

In [8]:
# Join all the deployments in one DataFrame
df = pd.DataFrame()
total_acoustic_time = 0
for idx in tqdm(metadata.index, total=len(metadata)):
    deployment_row = metadata.loc[idx]
    env_name = '%s_%s_env.nc' % (idx, deployment_row.deployment_name)
    env_path = processed_data_path.joinpath(env_name)
    deployment_file_name = '%s_%s.nc' % (idx, deployment_row.deployment_name)
    name = deployment_row['deployment_name']
    deployment = xarray.open_dataset(env_path)
    print(deployment)

    # Get the geometry
    geo_series = geopandas.GeoSeries(data=geopandas.points_from_xy(x=deployment['longitude'],
                                                                   y=deployment['latitude']),
                                     crs='EPSG:4326')

    # Eliminate the frequencies below MIN_FREQ and above MAX_FREQ
    deployment = deployment.sel(frequency=deployment.frequency[deployment.frequency < MAX_FREQ])
    deployment = deployment.sel(frequency=deployment.frequency[deployment.frequency > MIN_FREQ])
    deployment_duration = deployment.datetime.max() - deployment.datetime.min()
    total_acoustic_time += deployment_duration
    deployment = deployment[ENV_LABELS + [features_var]].dropna('id', 'any')
    clean_freqticks = deployment.frequency.values

    # Create a pandas df with all the wanted values
    values_arr = deployment[features_var].values
    df_deployment = pd.DataFrame(values_arr)
    df_deployment = df_deployment.astype(np.float16)
    for env in ENV_LABELS:
        df_deployment[env] = deployment[env].values

    df_deployment = geopandas.GeoDataFrame(df_deployment, geometry=geo_series)

    # Add the corresponding label by reading the csv with labels
    df_deployment['datetime'] = deployment.datetime
    df_deployment['filename'] = deployment.file_path.values
    df_deployment['label'] = 'unknown'

    for _, label_row in labels_data.iterrows():
        if deployment_file_name == label_row.filepath:
            mask_label = (df_deployment.datetime < (label_row.end_datetime -
                                                    datetime.timedelta(seconds=CHUNK_LENGTH))) & \
                         (df_deployment.datetime > label_row.start_datetime)
            if len(mask_label) > 0:
                df_deployment.loc[mask_label, 'label'] = label_row[label_column]

    df = pd.concat([df, df_deployment], ignore_index=True)

# print the total acoustic time
print('Total amount of time recorded %s h' % (total_acoustic_time.values / np.timedelta64(1, 'h')))

 50%|█████     | 2/4 [00:00<00:00,  9.34it/s]

<xarray.Dataset>
Dimensions:                 (id: 237, frequency: 31, band: 1, index: 237,
                             dim_0: 237)
Coordinates: (12/17)
    file_path               (id) object ...
  * id                      (id) int32 0 1 2 3 4 5 6 ... 231 232 233 234 235 236
    start_sample            (id) float64 ...
    end_sample              (id) float64 ...
    datetime                (id) datetime64[ns] ...
    hydrophone_sensitivity  (id) float64 ...
    ...                      ...
    low_freq                (band) int32 ...
    high_freq               (band) float64 ...
  * index                   (index) int64 0 1 2 3 4 5 ... 232 233 234 235 236
  * dim_0                   (dim_0) int64 0 1 2 3 4 5 ... 232 233 234 235 236
    lat                     (dim_0) float64 ...
    lon                     (dim_0) float64 ...
Data variables: (12/14)
    oct3                    (id, frequency) float64 ...
    rms                     (id, band) float64 ...
    sel                    

 75%|███████▌  | 3/4 [00:00<00:00,  8.35it/s]

<xarray.Dataset>
Dimensions:                 (id: 235, frequency: 31, band: 1, index: 235,
                             dim_0: 235)
Coordinates: (12/17)
    file_path               (id) object ...
  * id                      (id) int32 0 1 2 3 4 5 6 ... 229 230 231 232 233 234
    start_sample            (id) float64 ...
    end_sample              (id) float64 ...
    datetime                (id) datetime64[ns] ...
    hydrophone_sensitivity  (id) float64 ...
    ...                      ...
    low_freq                (band) int32 ...
    high_freq               (band) float64 ...
  * index                   (index) int64 0 1 2 3 4 5 ... 230 231 232 233 234
  * dim_0                   (dim_0) int64 0 1 2 3 4 5 ... 230 231 232 233 234
    lat                     (dim_0) float64 ...
    lon                     (dim_0) float64 ...
Data variables: (12/14)
    oct3                    (id, frequency) float64 ...
    rms                     (id, band) float64 ...
    sel                    

100%|██████████| 4/4 [00:00<00:00,  8.42it/s]

Total amount of time recorded 896.3058333333333 h


## Some data clean up

In [12]:
# Change the data types to save some computational power and memory
# Some operations
df = df.replace(['Civil twilight', 'Astronomical twilight', 'Nautical twilight'], ['Twilight', 'Twilight', 'Twilight'])
df['bathymetry'] = -1 * df['bathymetry']

# Categorical vars to category for efficient storage and processing
for env, env_type in vars_dtypes.items():
    df[env] = df[env].astype(env_type)

## Save the outputs to work on with the next script

In [14]:
# Filter the deployments to skip if there were any
np.save(processed_data_path.joinpath('used_freqticks.npy'), clean_freqticks)
df.to_pickle(processed_data_path.joinpath('df_complete.pkl'))